In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
age  = pd.read_csv('../data/cleaned_data/age.csv')
comm = pd.read_csv('../data/cleaned_data/comm.csv')

age  = age.set_index('county')
comm = comm.set_index('county')

In [ ]:
age

In [ ]:
comm

In [ ]:
data = pd.merge(comm,age,'outer',on = ['county','workers','drove_alone','carpooled',
                                      'pub_transit','state','year'])

In [ ]:
data

In [ ]:
pattern = r'mean'
keep = ['workers']
for col in data.columns:
    if re.search(pattern,col):
        keep.append(col)

In [ ]:
avg_comm = data[keep].groupby(data.index).mean()
avg_comm

In [ ]:
avg_comm.sort_values('mean_min', ascending = False)

In [ ]:
avg_comm.loc[avg_comm['pub_transit_mean_min'] < avg_comm['mean_min']].sort_values('pub_transit_mean_min')

In [ ]:
comm.loc['Lapeer County, Michigan']

In [ ]:
avg_comm.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

corr = avg_comm.corr(numeric_only = True)
# create a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Generate a mask for the upper triangle (since a correlation matrix is diagonally symmetric)
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, cmap=cmap, mask = mask, center=.4,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});